# Sciunit Cross-OS Reproducibility Demo on Chameleon

1. Create an **Ubuntu VM** to author a Sciunit experiment
2. Create a **Cent OS VM** to reproduce the experiment
3. Use SSH to interact with both machines
4. Demonstrate cross-OS reproducibility using Sciunit

This notebook focuses only on **infrastructure setup**.
Sciunit commands are executed directly on the VMs


In [ ]:
import chi, os, time, datetime
from chi import lease
from chi import server
from chi import context
from chi import network

context.version = "1.0"
context.choose_project()
context.choose_site(default="KVM@TACC")

username = os.getenv("USER")

print("proceed to next step")

In [ ]:
exp_name = "sciunit_cross_os_demo"

ubuntu_server_name = f"{exp_name}-ubuntu-{username}"
centOS_server_name = f"{exp_name}-centOS-{username}"
lease_name = f"{exp_name}-lease-{username}"

print("Experiment name:", exp_name)
print("proceed to next step")

In [ ]:
l = lease.Lease(
    lease_name,
    duration=datetime.timedelta(hours=6)
)

l.add_flavor_reservation(
    id=server.get_flavor_id("m1.small"),
    amount=2
)

l.submit(idempotent=True)
l.show()

print("proceed to next step")

In [ ]:
!mkdir -p ~/ssh_keys
!chmod 700 ~/ssh_keys
!ln -s /work/SciunitTrainingFiles-* /work/SciunitTrainingFiles

!mv /work/SciunitTrainingFiles/P_Key_NASA_DEMO.pem ~/ssh_keys/
!chmod 600 ~/ssh_keys/P_Key_NASA_DEMO.pem

!ls -ld ~/ssh_keys
!ls -l ~/ssh_keys/P_Key_NASA_DEMO.pem

print("proceed to next step")

In [ ]:
ubuntu_image = "CC-Ubuntu22.04"

ubuntu_vm = server.Server(
    name=ubuntu_server_name,
    image_name=ubuntu_image,
    flavor_name=l.get_reserved_flavors()[0].name,
    key_name="P_Key_NASA_DEMO"
)

ubuntu_vm.submit(idempotent=True)

print("proceed to next step")

In [ ]:
centOS_image = "CC-CentOS9-Stream-20230518"

centOS_vm = server.Server(
    name=centOS_server_name,
    image_name=centOS_image,
    flavor_name=l.get_reserved_flavors()[0].name,
    key_name="P_Key_NASA_DEMO"
)

centOS_vm.submit(idempotent=True)

print("proceed to next step")

In [ ]:
ubuntu_vm.associate_floating_ip()
print("[WAIT] Waiting 10 seconds for floating IP association...")
time.sleep(10)
ubuntu_ip = ubuntu_vm.get_floating_ip()
print("Ubuntu VM IP:", ubuntu_ip)

print("proceed to next step")

In [ ]:
centOS_vm.associate_floating_ip()
print("[WAIT] Waiting 10 seconds for floating IP association...")
time.sleep(10)
centOS_ip = centOS_vm.get_floating_ip()
print("centOS VM IP:", centOS_ip)

print("proceed to next step")

In [ ]:
sg_list = network.list_security_groups(name_filter="allow-ssh")

if sg_list:
    sg = sg_list[0]
else:
    sg = network.SecurityGroup({
        "name": "allow-ssh",
        "description": "Enable SSH access"
    })
    sg.add_rule("ingress", "tcp", 22)
    sg.submit()

ubuntu_vm.add_security_group(sg.id)
centOS_vm.add_security_group(sg.id)

print("proceed to next step")

In [ ]:
ubuntu_vm.check_connectivity()
centOS_vm.check_connectivity()

print("Both Ubuntu and centOS VMs are ready.")

print("proceed to next step")

In [ ]:
import paramiko

HOST = ubuntu_ip   # replace with your VM IP
USER = "cc"
KEY_PATH = "/home/nkmh5_missouri_edu/ssh_keys/P_Key_NASA_DEMO.pem"

ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())

ssh.connect(
    hostname=HOST,
    username=USER,
    key_filename=KEY_PATH,
    timeout=30,
    allow_agent=False,
    look_for_keys=False,
)

stdin, stdout, stderr = ssh.exec_command("whoami")
print(stdout.read().decode())

ssh.close()

print("proceed to next step")

In [ ]:
import paramiko

HOST = centOS_ip   # replace with your VM IP
USER = "cc"
KEY_PATH = "/home/nkmh5_missouri_edu/ssh_keys/P_Key_NASA_DEMO.pem"

ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())

ssh.connect(
    hostname=HOST,
    username=USER,
    key_filename=KEY_PATH,
    timeout=30,
    allow_agent=False,
    look_for_keys=False,
)

stdin, stdout, stderr = ssh.exec_command("whoami")
print(stdout.read().decode())

ssh.close()

print("proceed to next step")

In [ ]:
import paramiko

HOST = ubuntu_ip
USER = "cc"
KEY_PATH = "/home/nkmh5_missouri_edu/ssh_keys/P_Key_NASA_DEMO.pem"

SCRIPT_APT_PIP = r'''
set -e

echo "==== STEP 1: apt update ===="
sudo apt update -y
echo "OK: apt update"

echo "==== STEP 2: install python3-pip ===="
sudo apt install -y python3-pip
python3 -m pip --version
echo "OK: pip installed, proceed to next step"
exit 0
'''

ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())

ssh.connect(
    hostname=HOST,
    username=USER,
    key_filename=KEY_PATH,
    timeout=30,
    allow_agent=False,
    look_for_keys=False,
)

stdin, stdout, stderr = ssh.exec_command(
    f"bash -lc '{SCRIPT_APT_PIP}'",
    get_pty=True
)

for line in iter(stdout.readline, ""):
    print(line, end="")

err = stderr.read().decode()
if err:
    print(err)
    raise RuntimeError("APT / pip setup failed")

ssh.close()


In [ ]:
import paramiko

HOST = ubuntu_ip
USER = "cc"
KEY_PATH = "/home/nkmh5_missouri_edu/ssh_keys/P_Key_NASA_DEMO.pem"

SCRIPT_SCIUNIT = r'''
set -e

echo "==== STEP 4: clone sciunit ===="
cd ~
if [ ! -d "$HOME/sciunit" ]; then
  git clone https://github.com/radiant-systems-lab/sciunit.git
else
  echo "Sciunit repo already exists, skipping clone"
fi
echo "OK: sciunit ready"

echo "==== STEP 5: checkout nasa-demo ===="
cd ~/sciunit
git fetch
git checkout nasa-demo
echo " OK: nasa-demo branch"

echo "==== STEP 6: install cmake ===="
sudo apt install -y cmake
cmake --version
echo " OK: cmake installed"

echo "==== STEP 7: install sciunit ===="
pip install .
echo " OK: sciunit installed"

echo "==== STEP 8: update PATH (session only) ===="
export PATH="$HOME/.local/bin:$PATH"
echo "PATH now is: $PATH"
sciunit --version
echo " OK: PATH updated for this session, proceed to next step"
exit 0

'''

ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())

ssh.connect(
    hostname=HOST,
    username=USER,
    key_filename=KEY_PATH,
    timeout=30,
    allow_agent=False,
    look_for_keys=False,
)

stdin, stdout, stderr = ssh.exec_command(
    f"bash -lc '{SCRIPT_SCIUNIT}'",
    get_pty=True
)

for line in iter(stdout.readline, ""):
    print(line, end="")

err = stderr.read().decode()
if err:
    print(err)
    raise RuntimeError("Sciunit setup failed")

ssh.close()


In [ ]:
import paramiko

# --- CONFIGURATION ---
HOST = ubuntu_ip  # Replace with your actual IP
USER = "cc"
KEY_PATH = "/home/nkmh5_missouri_edu/ssh_keys/P_Key_NASA_DEMO.pem"

SETUP_SCRIPT = r'''
set -e

echo "==== STEP 1: clone training files repo ===="
cd ~
if [ ! -d "$HOME/SciunitTrainingFiles" ]; then
  git clone https://github.com/Nagarjuna-kandimalla/SciunitTrainingFiles.git
else
  echo "SciunitTrainingFiles already exists, skipping clone"
fi

echo "==== STEP 2: download HDF datasets ===="
cd ~/SciunitTrainingFiles
for file in GEOS5_202002.HDF IMERG_202002.HDF; do
    if [ ! -f "$file" ]; then
        wget -q "https://github.com/Nagarjuna-kandimalla/Python-Programming/raw/refs/heads/main/Geeks%20for%20geeks/$file"
        echo "Downloaded $file"
    else
        echo "$file already exists"
    fi
done

echo "==== STEP 3: download and install miniconda ===="
cd ~
if [ ! -f Miniconda3-latest-Linux-x86_64.sh ]; then
  wget -q https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
fi

if [ ! -d "$HOME/miniconda3" ]; then
  bash Miniconda3-latest-Linux-x86_64.sh -b -p $HOME/miniconda3
  echo " OK: miniconda installed"
fi

echo "==== STEP 5: load conda into current session ===="
source "$HOME/miniconda3/etc/profile.d/conda.sh"

echo "==== STEP 6: accept conda Terms of Service ===="
conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/main || true
conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/r || true

echo "==== STEP 7: create/activate python 3.6 env ===="
if ! conda env list | grep -q 'py36'; then
  conda create -y -n py36 python=3.6
fi
conda activate py36

echo "==== STEP 9: install scientific libraries ===="
conda install -y numpy pandas matplotlib scipy cartopy
pip install pyhdf

echo "==== STEP 11: verify imports ===="
python - << 'EOF'
import numpy, pandas, matplotlib, scipy, cartopy, pyhdf
print("ALL IMPORTS SUCCESSFUL")
EOF

echo "==== STEP 16: persist conda and environment ===="
# This ensures that whenever you SSH into Ubuntu, the shell is ready
if ! grep -q "conda activate py36" ~/.bashrc; then
    cat << 'EOF' >> ~/.bashrc

# >>> conda initialize >>>
source "$HOME/miniconda3/etc/profile.d/conda.sh"
conda activate py36
# <<< conda initialize <<<
EOF
    echo " OK: .bashrc updated for auto-activation"
else
    echo " OK: .bashrc already configured"
fi

cd ~/SciunitTrainingFiles
echo "ALL STEPS COMPLETED SUCCESSFULLY"
exit 0
'''

# ---------------- SSH EXECUTION ----------------

ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())

try:
    ssh.connect(
        hostname=HOST,
        username=USER,
        key_filename=KEY_PATH,
        timeout=30,
        allow_agent=False,
        look_for_keys=False,
    )

    # Use bash -s to read from stdin, avoiding long-string argument issues
    stdin, stdout, stderr = ssh.exec_command("bash -s", get_pty=True)
    
    # Write the script and CLOSE STDIN to signal the end of the command
    stdin.write(SETUP_SCRIPT)
    stdin.flush()
    stdin.close() 

    # Stream output live
    for line in iter(stdout.readline, ""):
        print(line, end="")

    # Wait for completion
    exit_status = stdout.channel.recv_exit_status()
    if exit_status == 0:
        print("\n--- UBUNTU SETUP FINISHED ---")
    else:
        print(f"\n--- FAILED with exit code {exit_status} ---")

finally:
    ssh.close()

### For Cento Machine

In [ ]:
import paramiko

HOST = centOS_ip
USER = "cc"
KEY_PATH = "/home/nkmh5_missouri_edu/ssh_keys/P_Key_NASA_DEMO.pem"

SCRIPT_SCIUNIT = r'''
set -e

echo "==== STEP 1: verify python & pip ===="
python3 --version
pip3 --version
echo " OK: python & pip verified"

echo "==== STEP 2: clone sciunit ===="
cd ~
if [ ! -d "$HOME/sciunit" ]; then
  git clone https://github.com/radiant-systems-lab/sciunit.git
else
  echo "Sciunit repo already exists"
fi
echo " OK: sciunit repo ready"

echo "==== STEP 3: checkout nasa-demo ===="
cd ~/sciunit
sudo dnf install -y cmake
git checkout nasa-demo
echo " OK: nasa-demo branch"

echo "==== STEP 4: install sciunit ===="
pip3 install --user .
echo " OK: sciunit installed"

echo "==== STEP 5: update PATH (session only) ===="
export PATH="$HOME/.local/bin:$PATH"
pip install --upgrade oauthlib
echo "PATH now is: $PATH"
sciunit --version
echo " OK: PATH updated for this session"

exit 0
'''

ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh.connect(HOST, username=USER, key_filename=KEY_PATH, timeout=30)

stdin, stdout, stderr = ssh.exec_command(
    f"bash -lc '{SCRIPT_SCIUNIT}'",
    get_pty=True
)

for line in iter(stdout.readline, ""):
    print(line, end="")

err = stderr.read().decode()
if err:
    print(err)
    raise RuntimeError("Sciunit install failed")

ssh.close()



In [ ]:
import paramiko

# --- CONFIGURATION ---
HOST = centOS_ip  # Replace with your variable
USER = "cc"
KEY_PATH = "/home/nkmh5_missouri_edu/ssh_keys/P_Key_NASA_DEMO.pem"

SCRIPT_CONDA = r'''
set -e

echo "==== STEP 1: clone training files repo ===="
cd ~
if [ ! -d "$HOME/SciunitTrainingFiles" ]; then
  git clone https://github.com/Nagarjuna-kandimalla/SciunitTrainingFiles.git
else
  echo "SciunitTrainingFiles already exists, skipping clone"
fi

echo "==== STEP 2: download HDF datasets ===="
cd ~/SciunitTrainingFiles
for file in GEOS5_202002.HDF IMERG_202002.HDF; do
    if [ ! -f "$file" ]; then
        wget -q "https://github.com/Nagarjuna-kandimalla/Python-Programming/raw/refs/heads/main/Geeks%20for%20geeks/$file"
        echo "Downloaded $file"
    else
        echo "$file already exists"
    fi
done

echo "==== STEP 3: download and install miniconda ===="
cd ~
if [ ! -f Miniconda3-latest-Linux-x86_64.sh ]; then
  wget -q https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
fi

if [ ! -d "$HOME/miniconda3" ]; then
  bash Miniconda3-latest-Linux-x86_64.sh -b -p $HOME/miniconda3
  echo " OK: miniconda installed"
fi

echo "==== STEP 5: load conda into current session ===="
source "$HOME/miniconda3/etc/profile.d/conda.sh"

echo "==== STEP 6: accept conda ToS ===="
conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/main || true
conda tos accept --override-channels --channel https://repo.anaconda.com/pkgs/r || true

echo "==== STEP 7: create/activate python 3.6 env ===="
if ! conda env list | grep -q 'py36'; then
  conda create -y -n py36 python=3.6
fi
conda activate py36

echo "==== STEP 9: install scientific stack ===="
conda install -y numpy pandas matplotlib scipy cartopy
pip install pyhdf

echo "==== STEP 10: verify imports ===="
python - << 'EOF'
import numpy, pandas, matplotlib, scipy, cartopy, pyhdf
print("ALL IMPORTS SUCCESSFUL")
EOF

echo "==== STEP 16: persist conda for future SSH sessions ===="
# We check if the activation line exists; if not, we append the block
if ! grep -q "conda activate py36" ~/.bashrc; then
    cat << 'EOF' >> ~/.bashrc

# >>> conda initialize >>>
source "$HOME/miniconda3/etc/profile.d/conda.sh"
conda activate py36
# <<< conda initialize <<<
EOF
    echo " OK: .bashrc updated for auto-activation"
else
    echo " OK: .bashrc already configured"
fi
cd ~/SciunitTrainingFiles
echo "CENTOS SETUP COMPLETED SUCCESSFULLY"
exit 0
'''

# ---------------- SSH EXECUTION ----------------

ssh = paramiko.SSHClient()
ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())

try:
    ssh.connect(
        hostname=HOST,
        username=USER,
        key_filename=KEY_PATH,
        timeout=30,
        allow_agent=False,
        look_for_keys=False,
    )

    # Use bash -s to pipe the script into standard input
    stdin, stdout, stderr = ssh.exec_command("bash -s", get_pty=True)
    
    # Send the script and close stdin so bash knows to start executing
    stdin.write(SCRIPT_CONDA)
    stdin.flush()
    stdin.close()

    # Stream output live
    for line in iter(stdout.readline, ""):
        print(line, end="")

    # Wait for the command to finish and get exit status
    exit_status = stdout.channel.recv_exit_status()
    
    if exit_status != 0:
        print(f"\nERROR: Script exited with status {exit_status}")
        err_content = stderr.read().decode()
        if err_content:
            print(f"Stderr: {err_content}")
    else:
        print("\n--- DONE ---")

finally:
    ssh.close()

### SSH commands for connecting to VM

In [ ]:


print("SSH into Ubuntu (Sciunit author):")
print(f"ssh -i ~/ssh_keys/P_Key_NASA_DEMO.pem cc@{ubuntu_ip}")

print("SSH into centOS (Sciunit reproducer ):")
print(f"ssh -i ~/ssh_keys/P_Key_NASA_DEMO.pem cc@{centOS_ip}")


## Cleanup 

In [ ]:
#ubuntu_vm.delete()
#centOS_vm.delete()
#l.delete()




_______________________________________________________________________________